<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
 

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [12]:
## install finrl library
#!pip install git+https://github.com/AI4Finance-Foundation/FinRL-Library.git

In [18]:
%reload_ext autoreload
%autoreload 2


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import os
import sys
sys.path.append('..')
from finrl import config
from finrl import config_tickers




%matplotlib inline
##from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from preprocess.default_preprocessors import FeatureEngineer, data_split 
from finrl.metaFinrl.preprocessor.CryptoDataReader import CryptoDataLoader
#from finrl.agents.stablebaselines3.models import DRLAgent
#from finrl.finrl_meta.data_processor import DataProcessor




from pprint import pprint


import itertools

<a id='1.4'></a>
## 2.4. Create Folders

In [19]:
from finrl.main import check_and_make_directories

check_and_make_directories([config.DATA_SAVE_DIR, config.TRAINED_MODEL_DIR, config.TENSORBOARD_LOG_DIR, config.RESULTS_DIR])

In [20]:
config.TRAINED_MODEL_DIR

'/mnt/f/financial_projects/Deep Reinforcement Learning Approaches on Stock Prediction/FinRL/MARKETS/ForexMarket/TRAINED_MODEL_DIR'

# Part 3: Download Data



-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


## 3.1 use yahoo finance api to get DOWJONES 

In [ ]:
df = YahooDownloader(start_date = datetime.datetime(2009, 1, 1),
                     end_date = datetime.datetime.now(),
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

In [6]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [7]:
df.shape

(100570, 8)

In [8]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2008-12-31,3.070357,3.133571,3.047857,2.602663,607541200,AAPL,2
1,2008-12-31,57.110001,58.220001,57.060001,43.587841,6287200,AMGN,2
2,2008-12-31,17.969999,18.750000,17.910000,14.852880,9625600,AXP,2
3,2008-12-31,41.590000,43.049999,41.500000,32.005890,5443100,BA,2
4,2008-12-31,43.700001,45.099998,43.700001,30.416967,6277400,CAT,2


In [12]:
data = df.copy()

## 3.2 read all the forexdata saved in csv  


In [5]:
import pandas as pd
import os

In [6]:
forex_symbols = pd.read_csv(os.path.join(config.DATA_SAVE_DIR,'forex_stocks_data.csv'))

In [7]:
forex_market_dataset = forex_symbols.drop(columns=['Unnamed: 0'])

In [9]:
forex_market_dataset = forex_symbols

In [8]:
forex_market_dataset.rename(columns={'symbol' : 'tic'}, inplace= True)

#### cleaning forex_symbols dataframe

In [25]:
dates = []
import datetime
for unixdate in forex_market_dataset['date']:
    utcdate = datetime.datetime.fromtimestamp(unixdate).strftime('%Y-%m-%dT%H:%M:%S')
    dates.append(utcdate)

TypeError: an integer is required (got type str)

In [32]:
forex_market_dataset.rename(columns= {'symbol' : 'tic'}, inplace= True)

In [ ]:
forex_market_dataset['date'] = dates

In [ ]:
forex_market_dataset

,close,high,low,open,s,date,volume,tic
0,3419.60000,3423.70000,3412.60000,3420.60000,ok,2022-07-01T10:30:00,463,OANDA:EU50_EUR
1,3438.00000,3444.90000,3401.90000,3419.60000,ok,2022-07-01T11:30:00,1691,OANDA:EU50_EUR
2,3456.00000,3462.90000,3428.90000,3440.00000,ok,2022-07-01T12:30:00,794,OANDA:EU50_EUR
3,3463.90000,3466.00000,3450.90000,3456.90000,ok,2022-07-01T13:30:00,685,OANDA:EU50_EUR
4,3441.00000,3465.90000,3439.90000,3464.90000,ok,2022-07-01T14:30:00,468,OANDA:EU50_EUR
...,...,...,...,...,...,...,...,...
490,0.69773,0.69848,0.69680,0.69834,ok,2022-07-29T20:30:00,3036,OANDA:AUD_USD
491,0.69771,0.69784,0.69713,0.69770,ok,2022-07-29T21:30:00,1886,OANDA:AUD_USD
492,0.69951,0.69952,0.69762,0.69771,ok,2022-07-29T22:30:00,1877,OANDA:AUD_USD
493,0.69864,0.69978,0.69837,0.69950,ok,2022-07-29T23:30:00,2004,OANDA:AUD_USD


In [ ]:
forex_market_dataset.sort_values(['date', 'tic'], ignore_index = True)
forex_market_dataset.drop(columns='s', inplace = True)
forex_market_dataset

,close,high,low,open,date,volume,tic
0,3419.60000,3423.70000,3412.60000,3420.60000,2022-07-01T10:30:00,463,OANDA:EU50_EUR
1,3438.00000,3444.90000,3401.90000,3419.60000,2022-07-01T11:30:00,1691,OANDA:EU50_EUR
2,3456.00000,3462.90000,3428.90000,3440.00000,2022-07-01T12:30:00,794,OANDA:EU50_EUR
3,3463.90000,3466.00000,3450.90000,3456.90000,2022-07-01T13:30:00,685,OANDA:EU50_EUR
4,3441.00000,3465.90000,3439.90000,3464.90000,2022-07-01T14:30:00,468,OANDA:EU50_EUR
...,...,...,...,...,...,...,...
490,0.69773,0.69848,0.69680,0.69834,2022-07-29T20:30:00,3036,OANDA:AUD_USD
491,0.69771,0.69784,0.69713,0.69770,2022-07-29T21:30:00,1886,OANDA:AUD_USD
492,0.69951,0.69952,0.69762,0.69771,2022-07-29T22:30:00,1877,OANDA:AUD_USD
493,0.69864,0.69978,0.69837,0.69950,2022-07-29T23:30:00,2004,OANDA:AUD_USD


## 3.3 : use finnhub to get crypto market data  

In [5]:
api_key='cbj22uqad3i2thcmtg80'
choosen_symbols = ['BTC', 'ETH', 'USDT', 'USDC', 'BNB', 'XRP', 'ADA', 'BUSD', 'SOL', 'DOT']

In [ ]:
data_loader = CryptoDataLoader(symbols_list= choosen_symbols, api_key=api_key)

In [ ]:
crypto_df = data_loader.load_crypto_candles()

In [ ]:
crypto_df

,c,h,l,o,s,t,v,symbol
0,0.06921,0.10000,0.01444,0.01444,ok,1586476800,3695792.1,BINANCE:SOLBNB
1,0.05773,0.07609,0.05568,0.06921,ok,1586563200,1792845.2,BINANCE:SOLBNB
2,0.06188,0.06589,0.05464,0.05756,ok,1586649600,1036908.3,BINANCE:SOLBNB
3,0.05171,0.06188,0.05160,0.06188,ok,1586736000,476803.6,BINANCE:SOLBNB
4,0.04272,0.05264,0.04040,0.05171,ok,1586822400,617685.5,BINANCE:SOLBNB
...,...,...,...,...,...,...,...,...
995,1.23700,1.33100,1.23700,1.31300,ok,1639612800,1450240.9,BINANCE:ADAUSDC
996,1.22200,1.25800,1.18300,1.23900,ok,1639699200,3451131.2,BINANCE:ADAUSDC
997,1.24300,1.26800,1.19900,1.21900,ok,1639785600,658339.6,BINANCE:ADAUSDC
998,1.24400,1.31100,1.24100,1.24200,ok,1639872000,1262579.4,BINANCE:ADAUSDC


In [ ]:
crypto_symbols_df = data_loader.cleansing_holcv_dataframes(crypto_df)

In [ ]:
dates = []
import datetime
for unixdate in crypto_symbols_df['date']:
    utcdate = datetime.datetime.fromtimestamp(unixdate).strftime('%Y-%m-%d')
    dates.append(utcdate)
crypto_symbols_df['date'] = dates

In [ ]:
data = crypto_symbols_df.copy()
data  = data_loader.crypto_clean_data(data)

,close,high,low,open,date,volume,tic
0,0.06921,0.10000,0.01444,0.01444,2020-04-10,3695792.1,BINANCE:SOLBNB
1,0.05773,0.07609,0.05568,0.06921,2020-04-11,1792845.2,BINANCE:SOLBNB
2,0.06188,0.06589,0.05464,0.05756,2020-04-12,1036908.3,BINANCE:SOLBNB
3,0.05171,0.06188,0.05160,0.06188,2020-04-13,476803.6,BINANCE:SOLBNB
4,0.04272,0.05264,0.04040,0.05171,2020-04-14,617685.5,BINANCE:SOLBNB
...,...,...,...,...,...,...,...
995,1.23700,1.33100,1.23700,1.31300,2021-12-16,1450240.9,BINANCE:ADAUSDC
996,1.22200,1.25800,1.18300,1.23900,2021-12-17,3451131.2,BINANCE:ADAUSDC
997,1.24300,1.26800,1.19900,1.21900,2021-12-18,658339.6,BINANCE:ADAUSDC
998,1.24400,1.31100,1.24100,1.24200,2021-12-19,1262579.4,BINANCE:ADAUSDC


In [ ]:
data.to_csv(os.path.join(config.DATA_SAVE_DIR,'Dataset.csv'))

NameError: name 'data' is not defined

In [6]:
data_name = 'Dataset.csv'
data  = pd.read_csv(os.path.join(config.DATA_SAVE_DIR,data_name))
data.drop(columns={'Unnamed: 0'}, inplace= True)

## 3.4 :use yahoofinance api to get s&p 500

In [ ]:
import requests
import bs4 as bs
import pickle

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers

In [ ]:
sp_tickers = save_sp500_tickers()
data = YahooDownloader(start_date = datetime.datetime(2016, 1, 1),
                     end_date = datetime.datetime.now(),
                     ticker_list = sp_tickers).fetch_data()

In [86]:
data = pd.DataFrame(data)
data.to_csv(os.path.join(config.DATA_SAVE_DIR,'sp500.csv'))

In [5]:
data = pd.read_csv(os.path.join(config.DATA_SAVE_DIR,'sp500.csv'))
data.drop(columns={'Unnamed: 0'}, inplace= True)

# alternative Part 3 : Load Data From CSV File




In [9]:
dataset_name = 'forex_data.csv'
forex_market_dataset = pd.read_csv(os.path.join(config.DATA_SAVE_DIR,dataset_name))

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [10]:
data = forex_market_dataset

In [11]:
data.drop(columns='Volume', inplace= True)

In [12]:
data.rename(columns={'Date': 'date','Adj Close': 'adj close', 'Close' : 'close', 'High' : 'high', 'Low': 'low', 'Open': 'open'}, inplace= True)

In [13]:
data.tic.unique()

array(['EURUSD=X', 'GBPUSD=X', 'USDJPY=X', 'AUDUSD=X', 'USDCAD=X',
       'USDCHF=X'], dtype=object)

In [14]:
!ls

NeurIPS_2018_PreprocessData.ipynb  NeurIPS_2018_backtest.ipynb
NeurIPS_2018_Train_Trade.ipynb	   mnt


In [26]:
from preprocess.default_preprocessors import FeatureEngineer, data_split 
from finrl import config



ModuleNotFoundError: No module named 'finrl.finrl_meta'

In [12]:
fe = FeatureEngineer(
                    use_technical_indicator = True,
                    tech_indicator_list =  config.INDICATORS,
                    use_vix= True ,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(data)


Index(['date', 'adj close', 'close', 'high', 'low', 'open', 'tic'], dtype='object')
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1007, 8)
Successfully added vix
Successfully added turbulence index


In [19]:
processed.tic.unique()

array(['AUDUSD=X', 'EURUSD=X', 'GBPUSD=X', 'USDCAD=X', 'USDCHF=X',
       'USDJPY=X'], dtype=object)

In [20]:
np.where(processed.isna() == True)

(array([], dtype=int64), array([], dtype=int64))

In [21]:
processed.dropna(axis=1)

,date,adj close,close,high,low,open,tic,macd,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2019-01-02,0.704791,0.704791,0.704821,0.698470,0.704722,AUDUSD=X,0.000014,100.000000,-66.666667,100.000000,0.704483,0.704483,23.219999,0.000000
1,2019-01-02,1.146171,1.146171,1.149700,1.134572,1.146132,EURUSD=X,-0.000070,0.000000,-66.666667,100.000000,1.147739,1.147739,23.219999,0.000000
2,2019-01-02,1.275429,1.275429,1.277335,1.258463,1.275234,GBPUSD=X,0.000036,100.000000,-66.666667,100.000000,1.274617,1.274617,23.219999,0.000000
3,2019-01-02,1.362540,1.362540,1.365960,1.356900,1.362500,USDCAD=X,0.000518,100.000000,66.666667,100.000000,1.350985,1.350985,23.219999,0.000000
4,2019-01-02,0.982000,0.982000,0.989250,0.979400,0.981800,USDCHF=X,-0.000034,0.000000,66.666667,100.000000,0.982755,0.982755,23.219999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,2022-12-29,1.062925,1.062925,1.067019,1.061233,1.062925,EURUSD=X,0.008925,60.167083,78.327051,49.139568,1.050900,1.022255,21.440001,6.166426
6032,2022-12-29,1.202848,1.202848,1.207584,1.201548,1.203297,GBPUSD=X,0.003764,52.732919,-46.930901,3.480165,1.211069,1.176042,21.440001,6.166426
6033,2022-12-29,1.359940,1.359940,1.360760,1.354450,1.359940,USDCAD=X,0.002138,52.738428,36.079932,8.181686,1.353157,1.357071,21.440001,6.166426
6034,2022-12-29,0.927710,0.927710,0.928720,0.921190,0.927710,USDCHF=X,-0.006936,40.654053,-97.756970,35.855458,0.938252,0.963402,21.440001,6.166426


In [13]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [14]:
processed_full.sort_values(['date','tic'],ignore_index=True)

,date,tic,adj close,close,high,low,open,macd,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2019-01-02,AUDUSD=X,0.704791,0.704791,0.704821,0.698470,0.704722,0.000014,100.000000,-66.666667,100.000000,0.704483,0.704483,23.219999,0.000000
1,2019-01-02,EURUSD=X,1.146171,1.146171,1.149700,1.134572,1.146132,-0.000070,0.000000,-66.666667,100.000000,1.147739,1.147739,23.219999,0.000000
2,2019-01-02,GBPUSD=X,1.275429,1.275429,1.277335,1.258463,1.275234,0.000036,100.000000,-66.666667,100.000000,1.274617,1.274617,23.219999,0.000000
3,2019-01-02,USDCAD=X,1.362540,1.362540,1.365960,1.356900,1.362500,0.000518,100.000000,66.666667,100.000000,1.350985,1.350985,23.219999,0.000000
4,2019-01-02,USDCHF=X,0.982000,0.982000,0.989250,0.979400,0.981800,-0.000034,0.000000,66.666667,100.000000,0.982755,0.982755,23.219999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,2022-12-29,EURUSD=X,1.062925,1.062925,1.067019,1.061233,1.062925,0.008925,60.167083,78.327051,49.139568,1.050900,1.022255,21.440001,6.166426
6032,2022-12-29,GBPUSD=X,1.202848,1.202848,1.207584,1.201548,1.203297,0.003764,52.732919,-46.930901,3.480165,1.211069,1.176042,21.440001,6.166426
6033,2022-12-29,USDCAD=X,1.359940,1.359940,1.360760,1.354450,1.359940,0.002138,52.738428,36.079932,8.181686,1.353157,1.357071,21.440001,6.166426
6034,2022-12-29,USDCHF=X,0.927710,0.927710,0.928720,0.921190,0.927710,-0.006936,40.654053,-97.756970,35.855458,0.938252,0.963402,21.440001,6.166426


In [ ]:
import os
processed_full.to_csv(os.path.join(config.DATA_SAVE_DIR,'full_preprocessed_data.csv'))